<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/15_optimization/050_Linear_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Linear Programming<br>선형계획법



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nl
import scipy.optimize as so



ref : 
* Wikipedia [link](https://en.wikipedia.org/wiki/Linear_programming)
* Stackoverflow [link](https://stackoverflow.com/questions/62571092/)
* Tips & Tricks on Linux, Matlab, vim, LaTex, etc [link](http://tipstrickshowtos.blogspot.com/2012/04/how-to-render-argmax-argmin-operator-in.html)



## Problem description<br>문제 설명



* Area of the farm 농장의 넓이 : $L = 10 (km^2)$
* Types of crops : wheat or rice<br>작물의 종류 : 밀 또는 쌀
* Available fertilizer 사용 가능한 비료의 양 : $F = 10 (kg)$
* Available pesticide 사용 가능한 살충제의 양 : $P = 5 (kg)$

|   | Wheat 밀 | rice 쌀 |
|:-----:|:-----:|:-----:|
| Needed Fertilizer per unit area $(kg/km^2)$<br>단위면적 당 필요 비료 양 $(kg/km^2)$ | $F_1$ | $F_2$ |
| Needed Pesticide per unit area $(kg/km^2)$<br>단위면적 당 필요 살충제 양 $(kg/km^2)$ | $P_1$ | $P_2$ |
| Selling price per unit area $(\$/km^2)$<br>단위면적 당 매출 $(\$/km^2)$ | $S_1$ | $S_2$ |
| Planting area $(km^2)$<br>재배 면적 $(km^2)$ | $x_1$ | $x_2$ |


* Under the constraints, what are the areas of wheat and rice maximizing the overall selling price?<br>제한조건 하에서 매출을 최대로 하는 밀과 쌀의 재배 면적?



$$
\underset{x_1, x_2}{\arg\max} \left(S_1 x_1 + S_2 x_2\right)
$$



subject to 제한조건



$$
\begin{align}
    x_1 + x_2 & \le L \\
    F_1 x_1 + F_2 x_2 & \le F \\
    P_1 x_1 + P_2 x_2 & \le P \\
    x_1, x_2 & \ge 0
\end{align}
$$



In matrix form 행렬 형태로는:



$$
\underset{x_1, x_2}{\arg\max} \begin{bmatrix} S_1 & S_2    \end{bmatrix}\begin{pmatrix} x_1 \\ x_2    \end{pmatrix}
$$



subject to 제한조건



$$
\begin{align}
    \begin{bmatrix}
        1 & 1 \\
        F_1 &  F_2 \\
        P_1 &  P_2 \\
    \end{bmatrix}
    \begin{pmatrix}
        x_1 \\
        x_2 
    \end{pmatrix} & \le 
    \begin{pmatrix}
        L \\
        F \\
        P
    \end{pmatrix} \\
    \begin{pmatrix}
        x_1 \\
        x_2 
    \end{pmatrix}& \ge 0
\end{align}
$$


## Parameters Example<br>매개변수 예



In [ ]:
L = 10



In [ ]:
F = 10 # 현재 확보된 비료 양
F1 = 2 # 밀 1 평방km 당 소모 비료 양
F2 = 3 # 쌀 1 평방km 당 소모 비료 양



In [ ]:
P = 5  # 현재 확보된 살충제 양
P1 = 2 # 밀 1 평방km 당 소모 살충제 양
P2 = 1 # 쌀 1 평방km 당 소모 살충제 양



In [ ]:
S1 = 20
S2 = 25



## Visualization 시각화



$$
\begin{align}
    x_1 + x_2 & \le L \\
    F_1 x_1 + F_2 x_2 & \le F \\
    P_1 x_1 + P_2 x_2 & \le P \\
    x_1, x_2 & \ge 0
\end{align}
$$



$$
\begin{align}
    x_2 & \le -x_1 + L \\
    x_2 & \le -\frac{F_1}{F_2} x_1 + \frac{F}{F_2} \\
    x_2 & \le -\frac{P_1}{P_2} x_1 + \frac{P}{P_2} \\
    x_1 & \ge 0 \\
    x_2 & \ge 0
\end{align}
$$


In [ ]:
def f_bound(x):
  return (-F1 * x + F) / F2

def p_bound(x):
  return (-P1 * x + P) / P2



In [ ]:
x1 = np.linspace(0, 2.5, 101)
x2 = np.linspace(0, 5, 101)

X1, X2 = np.meshgrid(x1, x2)

C = S1 * X1 + S2 * X2

# indicate regions beyond F & B bounds
C[X2 > f_bound(X1)] = np.nan
C[X2 > p_bound(X1)] = np.nan

plt.pcolor(X1, X2, C, shading="auto")

plt.plot(x1, f_bound(x1), label='F')
plt.plot(x1, p_bound(x1), label='P')

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

plt.legend(loc=0)
plt.title("$S_1 x_1 + S_2 x_2$")
plt.colorbar()

plt.contour(X1, X2, C, colors='m', linestyles='dashed')

plt.grid(True)



## `scipy.optimize.linprog()`



In [ ]:
c_T = -np.array((S1, S2))



In [ ]:
A_ub = np.array(
    (
        (1, 1),
        (F1, F2),
        (P1, P2),
    )
)



In [ ]:
b_ub = np.array(
    ((L, F, P),)
).T



Here, `.T` indicates transpose.<br>
여기서 `.T` 는 행과 열을 바꾸는 transpose



Bounds for $x_1$, $x_2$<br>
$x_1$, $x_2$ 의 범위



In [ ]:
bounds = (
    (0, 10),
    (0, 10),
)



In [ ]:
result = so.linprog(c_T, A_ub, b_ub, bounds=bounds)
result



In [ ]:
x1 = np.linspace(0, 2.5, 101)
x2 = np.linspace(0, 5, 101)

X1, X2 = np.meshgrid(x1, x2)

C = S1 * X1 + S2 * X2

# indicate regions beyond F & B bounds
C[X2 > f_bound(X1)] = np.nan
C[X2 > p_bound(X1)] = np.nan

plt.pcolor(X1, X2, C, shading="auto")


plt.plot(x1, f_bound(x1), label='F')
plt.plot(x1, p_bound(x1), label='P')

plt.plot(result.x[0], result.x[1], 'ro', label='Optimal')

plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

plt.legend(loc=0)
plt.title("$S_1 x_1 + S_2 x_2$")
plt.colorbar()

plt.contour(X1, X2, C, colors='m', linestyles='dashed')

plt.grid(True)

